In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# Defining the exponential decay function, which models the decay process
def exp_decay(time, N_0, k):
    return N_0 * np.exp(-k * time)

In [ ]:
# Load required packages
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# Exponential decay model:
# Expression(t) = Initial_amount * e^(-decay_rate * t)
def exponential_model(t, initial_amount, decay_rate):
    return initial_amount * np.exp(-decay_rate * t)

In [ ]:
# Function to estimate transcript half-life from replicate decay measurements
def estimate_half_life(expression_values, times):

    # Convert input values to numeric (invalid entries become NaN)
    expression_values = pd.to_numeric(expression_values, errors="coerce")

    # Remove zero or negative values to prevent issues with logarithms
    expression_values[expression_values <= 0] = np.nan

    # If no usable data remains, return NaN
    if expression_values.isna().all():
        return np.nan

    # Replace missing entries with the smallest positive observed value
    smallest_value = expression_values[expression_values > 0].min()
    expression_values.fillna(smallest_value, inplace=True)

    # Log-transform the expression levels
    log_values = np.log(expression_values)

    try:
        # Perform curve fitting with constraints to keep parameters positive
        fitted_params, _ = curve_fit(
            exp_decay,
            times,
            log_values,
            bounds=(0, np.inf)
        )

        initial_value, decay_constant = fitted_params

        # Compute half-life only if decay constant is valid
        if decay_constant > 0:
            return np.log(2) / decay_constant
        else:
            return np.nan

    except Exception:
        # If fitting fails, return NaN
        return np.nan

In [ ]:
# Read the tab-separated decay dataset into a DataFrame
data = pd.read_csv("DecayTimecourse.txt", sep="\t")

# Export the DataFrame to CSV format for later use
data.to_csv("DecayTimecourse.csv", index=False)

In [ ]:
# Read the previously saved CSV file back into a DataFrame
data = pd.read_csv("DecayTimecourse.csv")

# Create an empty list to collect calculated half-life values
half_life_results = []

# Specify the time intervals used in the decay experiment (in minutes)
time_points = np.array([0, 5, 10, 15, 20, 30, 40, 50, 60])

In [ ]:
# Loop through each transcript entry to compute half-life values
for idx, record in data.iterrows():

    gene_id = record["Time course #"]

    # Skip header-like entry if present
    if gene_id == "YORF":
        continue

    replicate_values = []

    # Process three experimental replicates
    for r in range(3):

        # Determine the column range corresponding to this replicate
        column_start = r * 9 + 2

        # Select expression measurements for this replicate
        replicate_series = record.iloc[column_start:column_start + 9]

        # Estimate half-life for the replicate
        hl_value = calculate_half_life(replicate_series, time_points)

        # Store only valid results
        if not np.isnan(hl_value):
            replicate_values.append(hl_value)

    # Compute mean half-life across available replicates
    if replicate_values:
        mean_half_life = np.mean(replicate_values)
        half_lives.append({
            "Transcript": gene_id,
            "Half_Life": mean_half_life
        })

In [ ]:
# Convert the collected half-life results into a DataFrame
results_df = pd.DataFrame(half_lives)

# Define filename and export results to CSV
file_name = "Half_Lives_Calculated.csv"
results_df.to_csv(file_name, index=False)

# Display calculated half-life values
print("Half-life results:")
for _, entry in results_df.iterrows():
    print(f"Gene: {entry['Transcript']} | Half-life: {entry['Half_Life']}")

print(f"\nResults successfully saved to: {file_name}")

# Read the saved results back into a DataFrame
results_df = pd.read_csv(file_name)

# Calculate 90th and 10th percentile cutoffs
upper_percentile = results_df["Half_Life"].quantile(0.9)
lower_percentile = results_df["Half_Life"].quantile(0.1)

# Select transcripts in the top and bottom 10%
long_lived_genes = results_df[results_df["Half_Life"] >= upper_percentile]
short_lived_genes = results_df[results_df["Half_Life"] <= lower_percentile]

# Extract gene names as lists
top_genes = long_lived_genes["Transcript"].tolist()
bottom_genes = short_lived_genes["Transcript"].tolist()

# Print gene lists
print("\nTop 10% (Most Stable Transcripts):")
print(top_genes)

print("\nBottom 10% (Least Stable Transcripts):")
print(bottom_genes)

Streaming output truncated to the last 5000 lines.
Gene: YJL159W | Half-life: 0.13610584710533546
Gene: YER177W | Half-life: 0.1356983273557203
Gene: YJL158C | Half-life: 0.13609089190741835
Gene: YLR056W | Half-life: 0.13713429323921345
Gene: YIL051C | Half-life: 0.13560480676759612
Gene: YBR078W | Half-life: 1.7715786825086053
Gene: YDL137W | Half-life: 0.13570993713443819
Gene: YDR050C | Half-life: 0.13692127010380678
Gene: YLR043C | Half-life: 0.2888126110997168
Gene: YDR077W | Half-life: 0.9736247139689749
Gene: YGR285C | Half-life: 0.1358414103492922
Gene: YBL077W | Half-life: 0.19005783251014685
Gene: YER072W | Half-life: 0.13719188678301258
Gene: YLR354C | Half-life: 0.13528715920042414
Gene: YPR035W | Half-life: 0.13653153046102046
Gene: YDR023W | Half-life: 0.1354673930796515
Gene: YEL026W | Half-life: 0.1377935936194099
Gene: YEL033W | Half-life: 0.548996080839615
Gene: YPL106C | Half-life: 0.13568241750883436
Gene: YHR007C | Half-life: 0.13535770534445407
Gene: YOR331C | Ha